# Generation of Hardware

This chapter shows how to generate the CGRA hardware with blocks.

## Setup

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

In [ ]:
import pillars.core.{ArchitectureHierarchy, Connect, HardwareGenerator}
import pillars.hardware.TopModule
import pillars.archlib.BlockImmediate
import pillars.archlib.TileLSUBlock
import pillars.core._
import pillars.archlib._
import pillars.core.{BlockTrait, OpEnum}
import pillars.core.OpEnum.OpEnum
import pillars.hardware.PillarsConfig

## `Example1`: Top Module with a single block (i.e., `BlockImmediate`).

### Step1: Create a block. 

This part is mentioned in Chapter3 of our tutorial.
 
### Step2: Construct the architecture of the top module

In [ ]:
val arch = new ArchitectureHierarchy()

val blockImm = new BlockImmediate("blockImm")
arch.addBlock(blockImm)

arch.addInPorts(Array("in0", "in1"))
arch.addOutPorts(Array("out0"))

arch.addConnect(arch.term("in0") -> blockImm/"in0")
arch.addConnect(arch.term("in1") -> blockImm/"in1")
arch.addConnect(blockImm/"out0" -> arch.term("out0"))

arch.init()

### Step3: Construct the top module using `HardwareGenerator`

In [ ]:
val hardwareGenerator = new HardwareGenerator(arch, new Connect(arch.connectArray))

val topDesign = () => new TopModule(
    hardwareGenerator.pillarsModuleInfo, hardwareGenerator.connectMap,
    hardwareGenerator.regionList, blockImm.aluParams(0)
)

println((new chisel3.stage.ChiselStage).emitVerilog(topDesign()))
//visualize(topDesign)

### *Generate a CGRA without synchronizers and schedule controllers*

In [ ]:
//If only you need configuration controller
PillarsConfig.LOG_SCHEDULE_SIZE = 0
PillarsConfig.SKEW_REGISTER_NUM = -1
PillarsConfig.update_auxiliary()
//visualize(topDesign)

## `Example2`: Generate a CGRA with a nest block and a simple blok

### Step1: Declare SimpleBlock and NestBlock created in Chapter3

In [ ]:
class SimpleBlock(name: String, isRegion: Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0","input_1"))
    addOutPorts(Array("out_0"))
    
    if(isRegion){
        setConfigRegion()
    }
    
    // Initialize ALU supporting ADD/SUB
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB)
    val supBypass = false 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, List(32))
    alu0.addInPorts(Array("input_A", "input_B"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)
    
    // Initialize internal connections
    addConnect(term("input_0") -> alu0 / "input_A")
    addConnect(term("input_1") -> alu0 / "input_B")
    addConnect(alu0 / "out_0" -> term("out_0"))
}

class NestBlock(name: String, isRegion:Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0", "input_1"))
    addOutPorts(Array("out_0"))

     if(isRegion){
        setConfigRegion()
    }
    
    // A multiplexer that can choose a data source for the port "inputA" of the ALU.
    val muxParams = List(2, 32) // 2-input 32-bit width
    val mux0 = new ElementMux(name+"_mux", muxParams)
    mux0.addInPorts(Array("input_0", "input_1"))
    mux0.addOutPorts(Array("out_0"))
    addElement(mux0)
    
    // An ALU that can perform some operations.
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB, OpEnum.AND, OpEnum.OR, OpEnum.XOR, OpEnum.MUL)
    val supBypass = true 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, aluParams)
    
    alu0.addInPorts(Array("inputA", "inputB"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)
    
    // A const unit connected to the port "inputB" of ALU.
    val constParams = List(32) // 32 bit width
    val const0 = new ElementConst(name+"_const0", constParams)
    const0.addOutPorts(Array("out_0"))
    addElement(const0)
    
    // A simple sub-block with 2 input ports and 1 output port.
    val subBLock = new SimpleBlock(name+"_subBlock")
    addBlock(subBLock)

    // Interconnection inside this block.
    addConnect(term("input_0") -> mux0 / "input_0")
    addConnect(term("input_1") -> mux0 / "input_1")
    addConnect(mux0 / "out_0" -> alu0 / "inputA")
    addConnect(const0 / "out_0" -> alu0 / "inputB")
    addConnect(term("input_1") -> subBLock / "input_0")
    addConnect(alu0 / "out_0" -> subBLock / "input_1")
    addConnect(subBLock / "out_0" -> term("out_0"))
}

### Step2: Connect the two blocks in architecture.

In [ ]:
    val inputPort = 3
    val outputPort = 2
    val dataWidth = 32

    //Initialize the top block.
    val arch = new ArchitectureHierarchy()
    arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
    arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)
    val block = new NestBlock("Block0", isRegion = true)
    arch.addBlock(block)

    val block1 = new SimpleBlock("Block1", isRegion = true)
    arch.addBlock(block1)

    arch.addConnect(arch.term(s"input_0") -> block / s"input_0")
    arch.addConnect(arch.term(s"input_1") -> block / s"input_1")
    arch.addConnect(block / s"out_0" -> block1 / s"input_0")
    arch.addConnect(arch.term(s"input_2") -> block1 / s"input_1")
    arch.addConnect(block / s"out_0" -> arch.term(s"out_0"))
    arch.addConnect(block1 / s"out_0" -> arch.term(s"out_1"))

    arch.init()

### Step3: Generate the hardware.

In [ ]:
val connect = new Connect(arch.connectArray)
val hardwareGenerator = new HardwareGenerator(arch, connect)
val topDesign = () => new TopModule(hardwareGenerator.pillarsModuleInfo,
    hardwareGenerator.connectMap, hardwareGenerator.regionList, dataWidth)
    
//visualize(topDesign)

## `Example3`: Use block library to create a CGRA

In [ ]:
    val rowNum = 2
    val colNum = 2
    val inputPort = 1
    val outputPort = 1
    val dataWidth = 32

    //Initialize the top block.
    val arch = new ArchitectureHierarchy()
    arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
    arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

    val tile = new TileLSUBlock("tile_0", colNum, rowNum, inputPort, outputPort,
      useMuxBypass = false, complex = true, isToroid = false, useCounter = true, dataWidth = dataWidth)
    arch.addBlock(tile)

    (0 until inputPort).foreach(i =>
      arch.addConnect(arch.term(s"input_$i") -> tile / s"input_$i"))
    (0 until outputPort).foreach(i =>
      arch.addConnect(tile / s"out_$i" -> arch.term(s"out_$i")))
    arch.init()

    val connect = new Connect(arch.connectArray)
    val hardwareGenerator = new HardwareGenerator(arch, connect)
    val topDesign = () => new TopModule(hardwareGenerator.pillarsModuleInfo,
      hardwareGenerator.connectMap, hardwareGenerator.regionList, dataWidth)
      
    //visualize(topDesign)